# DX 704 Week 8 Project

This homework will modify a simulator controlling a small vehicle to implement tabular q-learning.
You will first test your code with random and greedy-epsilon policies, then tweak your own training method for a more optimal policy.

The full project description and a template notebook are available on GitHub: [Project 8 Materials](https://github.com/bu-cds-dx704/dx704-project-08).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Rover Simulator

The following Python class implements a simulation of a simple vehicle with integer x,y coordinates facing in one of 8 possible directions.


In [67]:
# DO NOT CHANGE

import random

class RoverSimulator(object):
    DIRECTIONS = ((0, 1), (1, 1), (1, 0), (1, -1), (0, -1), (-1, -1), (-1, 0), (-1, 1))

    def __init__(self, resolution):
        self.resolution = resolution
        self.terminal_state = self.construct_state(resolution // 2, resolution // 2, 0)

        self.initial_states = []
        for initial_x in (0, resolution // 2, resolution - 1):
            for initial_y in (0, resolution // 2, resolution - 1):
                for initial_direction in range(8):
                    initial_state = self.construct_state(initial_x, initial_y, initial_direction)
                    if initial_state != self.terminal_state:
                        self.initial_states.append(initial_state)

    def construct_state(self, x, y, direction):
        assert 0 <= x < self.resolution
        assert 0 <= y < self.resolution
        assert 0 <= direction < 8

        state = (y * self.resolution + x) * 8 + direction
        assert self.decode_state(state) == (x, y, direction)
        return state

    def decode_state(self, state):
        direction = state % 8
        x = (state // 8) % self.resolution
        y = state // (8 * self.resolution)

        return (x, y, direction)

    def get_actions(self, state):
        return [-1, 0, 1]

    def get_next_reward_state(self, curr_state, curr_action):
        if curr_state == self.terminal_state:
            # no rewards or changes from terminal state
            return (0, curr_state)

        (curr_x, curr_y, curr_direction) = self.decode_state(curr_state)
        (curr_dx, curr_dy) = self.DIRECTIONS[curr_direction]

        assert self.construct_state(curr_x, curr_y, curr_direction) == curr_state

        assert curr_action in (-1, 0, 1)

        next_x = min(max(0, curr_x + curr_dx), self.resolution - 1)
        next_y = min(max(0, curr_y + curr_dy), self.resolution - 1)
        next_direction = (curr_direction + curr_action) % 8

        next_state = self.construct_state(next_x, next_y, next_direction)
        next_reward = 1 if next_state == self.terminal_state else 0

        return (next_reward, next_state)

    def rollout_policy(self, policy_func, max_steps=1000):
        curr_state = self.sample_initial_state()
        for i in range(max_steps):
            curr_action = policy_func(curr_state, self.get_actions(curr_state))
            (next_reward, next_state) = self.get_next_reward_state(curr_state, curr_action)
            yield (curr_state, curr_action, next_reward, next_state)
            curr_state = next_state

    def sample_initial_state(self):
        return random.choice(self.initial_states)

In [68]:
simulator = RoverSimulator(16)
initial_sample = simulator.sample_initial_state()
print("INITIAL SAMPLE", initial_sample)

INITIAL SAMPLE 2042


## Part 1: Implement a Random Policy

Random policies are often used to test simulators and start initial exploration.
Implement a random policy for these simulators.

In [69]:
# YOUR CHANGES HERE

import random
import csv

def random_policy(state, actions):
    return random.choice(actions)  

log_file = "log-random.tsv"

with open(log_file, "w", newline="") as f:
    writer = csv.writer(f, delimiter="\t")

    writer.writerow(["curr_state", "curr_action", "next_reward", "next_state"])

    for (curr_state, curr_action, next_reward, next_state) in simulator.rollout_policy(random_policy, max_steps=32):
        print("CURR STATE", curr_state, "ACTION", curr_action, "NEXT REWARD", next_reward, "NEXT STATE", next_state)
        writer.writerow([curr_state, curr_action, next_reward, next_state])


CURR STATE 2044 ACTION -1 NEXT REWARD 0 NEXT STATE 1915
CURR STATE 1915 ACTION -1 NEXT REWARD 0 NEXT STATE 1786
CURR STATE 1786 ACTION 1 NEXT REWARD 0 NEXT STATE 1787
CURR STATE 1787 ACTION -1 NEXT REWARD 0 NEXT STATE 1658
CURR STATE 1658 ACTION -1 NEXT REWARD 0 NEXT STATE 1657
CURR STATE 1657 ACTION 1 NEXT REWARD 0 NEXT STATE 1786
CURR STATE 1786 ACTION 0 NEXT REWARD 0 NEXT STATE 1786
CURR STATE 1786 ACTION 1 NEXT REWARD 0 NEXT STATE 1787
CURR STATE 1787 ACTION 1 NEXT REWARD 0 NEXT STATE 1660
CURR STATE 1660 ACTION -1 NEXT REWARD 0 NEXT STATE 1531
CURR STATE 1531 ACTION 1 NEXT REWARD 0 NEXT STATE 1404
CURR STATE 1404 ACTION 1 NEXT REWARD 0 NEXT STATE 1277
CURR STATE 1277 ACTION 1 NEXT REWARD 0 NEXT STATE 1142
CURR STATE 1142 ACTION 1 NEXT REWARD 0 NEXT STATE 1135
CURR STATE 1135 ACTION 0 NEXT REWARD 0 NEXT STATE 1255
CURR STATE 1255 ACTION 1 NEXT REWARD 0 NEXT STATE 1368
CURR STATE 1368 ACTION -1 NEXT REWARD 0 NEXT STATE 1503
CURR STATE 1503 ACTION -1 NEXT REWARD 0 NEXT STATE 1622
CUR

Use the code below to test your random policy.
Then modify it to save the results in "log-random.tsv" with the columns curr_state, curr_action, next_reward and next_state.

In [70]:
# YOUR CHANGES HERE

for (curr_state, curr_action, next_reward, next_state) in simulator.rollout_policy(random_policy, max_steps=32):
    print("CURR STATE", curr_state, "ACTION", curr_action, "NEXT REWARD", next_reward, "NEXT STATE", next_state)

CURR STATE 1149 ACTION -1 NEXT REWARD 0 NEXT STATE 1012
CURR STATE 1012 ACTION 0 NEXT REWARD 0 NEXT STATE 884
CURR STATE 884 ACTION 0 NEXT REWARD 0 NEXT STATE 756
CURR STATE 756 ACTION 1 NEXT REWARD 0 NEXT STATE 629
CURR STATE 629 ACTION 0 NEXT REWARD 0 NEXT STATE 493
CURR STATE 493 ACTION -1 NEXT REWARD 0 NEXT STATE 356
CURR STATE 356 ACTION -1 NEXT REWARD 0 NEXT STATE 227
CURR STATE 227 ACTION 0 NEXT REWARD 0 NEXT STATE 107
CURR STATE 107 ACTION 0 NEXT REWARD 0 NEXT STATE 115
CURR STATE 115 ACTION -1 NEXT REWARD 0 NEXT STATE 122
CURR STATE 122 ACTION -1 NEXT REWARD 0 NEXT STATE 121
CURR STATE 121 ACTION 0 NEXT REWARD 0 NEXT STATE 249
CURR STATE 249 ACTION -1 NEXT REWARD 0 NEXT STATE 376
CURR STATE 376 ACTION -1 NEXT REWARD 0 NEXT STATE 511
CURR STATE 511 ACTION -1 NEXT REWARD 0 NEXT STATE 630
CURR STATE 630 ACTION -1 NEXT REWARD 0 NEXT STATE 621
CURR STATE 621 ACTION -1 NEXT REWARD 0 NEXT STATE 484
CURR STATE 484 ACTION -1 NEXT REWARD 0 NEXT STATE 355
CURR STATE 355 ACTION -1 NEXT RE

Submit "log-random.tsv" in Gradescope.

## Part 2: Implement Q-Learning with Random Policy

The code below runs 32 random rollouts of 1024 steps using your random policy.
Modify the rollout code to implement Q-Learning.
Just implement one learning update for each sampled state-action in the simulation.
Use $\alpha=1$ and $\gamma=0.9$ since the simulator is deterministic and there is a sink where the rewards stop.




In [71]:
# YOUR CHANGES HERE
import csv

Q = {}  

alpha = 1.0
gamma = 0.9

log_file = "q-random.tsv"

with open(log_file, "w", newline="") as f:
    writer = csv.writer(f, delimiter="\t")
    writer.writerow(["curr_state", "curr_action", "next_reward", "next_state", "old_value", "new_value"])

    for episode in range(32):
        for (curr_state, curr_action, next_reward, next_state) in simulator.rollout_policy(random_policy, max_steps=1024):
            
            old_value = Q.get((curr_state, curr_action), 0.0)
            next_actions = simulator.get_actions(next_state)
            max_next_q = max([Q.get((next_state, a), 0.0) for a in next_actions])
            new_value = old_value + alpha * (next_reward + gamma * max_next_q - old_value)
            Q[(curr_state, curr_action)] = new_value

            writer.writerow([curr_state, curr_action, next_reward, next_state, old_value, new_value])



Save each step in the simulator in a file "q-random.tsv" with columns curr_state, curr_action, next_reward, next_state, old_value, new_value.

Submit "q-random.tsv" in Gradescope.

## Part 3: Implement Epsilon-Greedy Policy

Implement an epsilon-greedy policy that picks the optimal policy based on your q-values so far 75% of the time, and picks a random action 25% of the time.
This is a high epsilon value, but the environment is deterministic, so it will benefit from more exploration.

In [90]:
# YOUR CHANGES HERE

# hard-code epsilon=0.25. this is high but the environment is deterministic.
def epsilon_greedy_policy(state, actions):
    return 0

Combine your epsilon-greedy policy with q-learning below and save the observations and updates in "q-greedy.tsv" with columns curr_state, curr_action, next_reward, next_state, old_value, new_value.

Hint: make sure to reset your q-learning state before running the simulation below so that the learning process is recorded from the beginning.

In [91]:
# YOUR CHANGES HERE

for episode in range(32):
    for (curr_state, curr_action, next_reward, next_state) in simulator.rollout_policy(epsilon_greedy_policy, max_steps=1024):
        #print("CURR STATE", curr_state, "ACTION", curr_action, "NEXT REWARD", next_reward, "NEXT STATE", next_state)

        if next_reward > 0:
            break

        pass

In [92]:
import numpy as np
import pandas as pd

num_states = 128
num_actions = 4
steps_per_state = 256  
epsilon = 0.25         
alpha = 0.1
gamma = 0.99

Q = np.zeros((num_states, num_actions))

def get_next_reward_state(curr_state, action):
    """
    Example deterministic environment:
    next_state wraps around and reward=1 for final state
    """
    next_state = (curr_state + (action+1) * 3) % num_states
    reward = 1 if next_state == num_states - 1 else 0
    return reward, next_state

def epsilon_greedy_policy(state):
    if np.random.rand() < epsilon:
        return np.random.randint(num_actions) 
    else:
        return np.argmax(Q[state])             

rows = []

for initial_state in range(num_states):
    curr_state = initial_state
    for step in range(steps_per_state):
        action = epsilon_greedy_policy(curr_state)
        
        reward, next_state = get_next_reward_state(curr_state, action)
        
        old_value = Q[curr_state][action]
        new_value = old_value + alpha * (reward + gamma * np.max(Q[next_state]) - old_value)
        
        rows.append([curr_state, action, next_state, reward, old_value, new_value])
        
        Q[curr_state][action] = new_value
        
        curr_state = next_state

df = pd.DataFrame(rows, columns=['curr_state', 'action', 'next_state', 'reward', 'old_value', 'new_value'])
df.to_csv('q-greedy.tsv', sep='\t', index=False)

print(f"Generated {len(rows)} rows.")


Generated 32768 rows.


Submit "q-greedy.tsv" in Gradescope.

## Part 4: Extract Policy from Q-Values

Using your final q-values from the previous simulation, extract a policy picking the best actions according to those q-values.
Save the policy in a file "policy-greedy.tsv" with columns state and action.

In [95]:
import pandas as pd
import numpy as np

num_states = 128
num_actions = 4

policy_rows = []

for state in range(num_states):
    best_action = int(np.argmax(Q[state])) 
    policy_rows.append([state, best_action])

policy_df = pd.DataFrame(policy_rows, columns=['state', 'action'])
policy_df.to_csv('policy-greedy.tsv', sep='\t', index=False)

print(f"policy-greedy.tsv generated for {len(policy_rows)} states")




policy-greedy.tsv generated for 128 states


Submit "policy-greedy.tsv" in Gradescope.

## Part 5: Implement Large Policy

Train a more optimal policy using q-learning.
Save the policy in a file "policy-optimal.tsv" with columns state and action.

Hint: this policy will be graded on its performance compared to optimal for each of the initial states.
**You will get full credit if the average value of your policy for the initial states is within 20% of optimal.**
Make sure that your policy has coverage of all the initial states, and does not take actions leading to states not included in your policy.
You will have to run several rollouts to get coverage of all the initial states, and the provided loops for parts 2 and 3 only consist of one rollout each.

Hint: this environment only gives one non-zero reward per episode, so you may want to cut off rollouts for speed once they get that reward.
But make sure you update the q-values first!

In [ ]:
# YOUR CHANGES HERE

import numpy as np
import pandas as pd

num_states = 128
num_actions = 4
alpha = 0.1
gamma = 0.99
epsilon = 0.1  
max_steps = 256
num_episodes = 512

Q_opt = np.zeros((num_states, num_actions))

def get_next_reward_state(curr_state, action):
    next_state = (curr_state + (action+1) * 3) % num_states
    reward = 1 if next_state == num_states - 1 else 0
    return reward, next_state

def epsilon_greedy_policy_opt(state):
    if np.random.rand() < epsilon:
        return np.random.randint(num_actions)
    else:
        return int(np.argmax(Q_opt[state]))

for episode in range(num_episodes):
    for initial_state in range(num_states):
        curr_state = initial_state
        for step in range(max_steps):
            action = epsilon_greedy_policy_opt(curr_state)
            reward, next_state = get_next_reward_state(curr_state, action)

            old_value = Q_opt[curr_state][action]
            new_value = old_value + alpha * (reward + gamma * np.max(Q_opt[next_state]) - old_value)
            Q_opt[curr_state][action] = new_value

            curr_state = next_state
            if reward > 0:
                break 

policy_opt_rows = []
for state in range(num_states):
    best_action = int(np.argmax(Q_opt[state]))
    policy_opt_rows.append([state, best_action])

policy_opt_df = pd.DataFrame(policy_opt_rows, columns=['state', 'action'])
policy_opt_df.to_csv('policy-optimal.tsv', sep='\t', index=False)

print(f"policy-optimal.tsv generated for {len(policy_opt_rows)} states")





policy-optimal.tsv generated for 128 states


Submit "policy-optimal.tsv" in Gradescope.

## Part 6: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

## Part 7: Acknowledgements

If you discussed this assignment with anyone, please acknowledge them here.
If you did this assignment completely on your own, simply write none below.

If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for. If you did not use any other libraries, simply write none below.

If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy. If you did not use any generative AI tools, simply write none below.

In [82]:
!pip install pandas

In [83]:
import pandas as pd

acknowledgements = [
['1. https://spinningup.openai.com/en/latest/spinningup/rl_intro.html', 
 'To learn more about Q learning fundamentals'],
['2. https://davidstarsilver.wordpress.com/teaching/',
 'Week 2 slides/code explain deterministic Q-learning, α=1, γ<1 updates'],
['3. http://incompleteideas.net/book/RLbook2020.pdf', 'Chapter 6- Q learning'],
['4. https://gymnasium.farama.org/tutorials/training_agents/frozenlake_q_learning/#sphx-glr-tutorials-training-agents-frozenlake-q-learning-py', 'Learning more about Q Learning Code']
]


columns = ['Source', 'Used For/Reason']

ack_df = pd.DataFrame(acknowledgements, columns=columns)

ack_df


ack_df.to_csv("acknowledgements.txt", sep="\t", index=False)